Хотел обучить какую нибудь простую модельку, не вышло, работает слишком плохо, не успел придумать ничего другого.

Идея была следующая - обучить 2 модельки, которые предсказывают отдельно ask и bid цены. Если следующая bid цена будет выше предыдущей ask цены (при условии что разница цен больше 2 комиссий), то мы покупаем на данном шаге. Если наоборот, то продаем.
Идем окном размера N по ценам и подаем в модель $[x_{k-N}, ..., x_{k-1}]$ и предсказываем $x_k$

In [1]:
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader, Dataset, TensorDataset
from typing import Any, List, Tuple
from sklearn.metrics import mean_squared_error
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [5]:
import json
with open('bitmex.json') as f:
    ask = np.array(json.load(f))
len(ask)

6036368

In [57]:
from tqdm import tqdm
def run_epoch(model, dataloader, criterion, device='cpu', optimizer=None, do_train=True, scheduler=None):
    loss_log, preds, gts = [], [], []
    prefix = ['Val', 'Train'][do_train] + ' - {}'
    model.train(do_train)

    for x_batch, y_batch in tqdm(dataloader, position=0):
        data = x_batch.to(device)
        target = y_batch.to(device)

        if do_train:
            optimizer.zero_grad()

        with torch.inference_mode(not do_train):
            output = model(data)
            loss = criterion(output, target).cpu()
            
        preds.append(output.detach().cpu())
        gts.append(y_batch.detach().cpu())
        loss_log.append(loss.item())

        if not do_train:
            continue
    
        loss.backward()
        optimizer.step()

        if scheduler:
            scheduler.step()
    return torch.cat(gts), torch.cat(preds), loss_log
    
def train(model, dataloaders, optimizer, criterion, n_epochs, device='cpu', scheduler=None, submit=False):
    prefix = 'Full val - {}'
    for epoch in tqdm(range(n_epochs), position=0):
        print("Epoch {0} of {1}".format(epoch, n_epochs))

        train_targets, train_preds, train_loss = run_epoch(
            model=model,
            dataloader=dataloaders['train'],
            criterion=criterion, 
            optimizer=optimizer,
            do_train=True,
            scheduler=scheduler,
            device=device,
        )
        print("Train MSE: ", mean_squared_error(train_targets, train_preds), end='   ')
        if not submit:
            val_targets, val_preds, val_loss = run_epoch(
                model=model,
                dataloader=dataloaders['val'],
                criterion=criterion, 
                optimizer=None,
                do_train=False,
                scheduler=None,
                device=device,
            )
            print("Val MSE: ", mean_squared_error(val_targets, val_preds))

def inference(model, dataloader, device='cpu'):
    preds = []
    model.eval()
    for x_batch in tqdm(dataloader):
        data = x_batch[0].to(device)
        with torch.inference_mode():
            output = model(data)
            preds.append(output.detach().cpu())
            
    return torch.cat(preds)

In [58]:
class CustomDataset(Dataset):
    """
    Dataset for sampling stock data 
    Attributes:
        stock: List of stock prices.
        window: Number of previous prices to predict next.
    """
    def __init__(self, stock, window=10) -> None:
        self.stock = stock
        self.window = window
        
    def __len__(self) -> int:
        return len(self.stock) - self.window
    
    def __getitem__(self, idx) -> Tuple[Tensor, Tensor]:
        prices = torch.tensor(self.stock[idx:idx+self.window]).float()
        target = torch.tensor(self.stock[idx+self.window]).float()
        prices = prices.unsqueeze(1)
        return prices, target

Возьмем каждое 20 значение, разделим на train и val

In [99]:
ask_cp = ask[::20]

train_ask = ask_cp[:int(len(ask_cp)*0.8)]
val_ask = ask_cp[int(len(ask_cp)*0.8):]

train_ask_dataset = CustomDataset(train_ask)
val_ask_dataset = CustomDataset(val_ask)

train_ask_dataloader = DataLoader(
    train_ask_dataset,
    batch_size=64,
    pin_memory=True,
    num_workers=2,
    shuffle=False
)
val_ask_dataloader = DataLoader(
    val_ask_dataset, 
    batch_size=64, 
    pin_memory=True, 
    num_workers=2, 
    shuffle=False
)

ask_dataloaders = {'train': train_ask_dataloader, 'val': val_ask_dataloader}

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [100]:
class AskNet(nn.Module):
    def __init__(self):
        super(AskNet, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=16, num_layers=2, batch_first=True)
        self.linear = nn.Linear(16, 1)
        
    def forward(self, x):
        x, _  = self.lstm(x)
        out = self.linear(x[:,-1,:]).squeeze()
        return out

In [102]:
n_epochs = 30
ask_model = AskNet()
ask_model.to(device)
optimizer = torch.optim.Adam(ask_model.parameters(), lr=1e-2)
criterion = nn.MSELoss()
train(ask_model, ask_dataloaders, optimizer, criterion, n_epochs=n_epochs, device=device)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0 of 30


100%|██████████| 3773/3773 [00:25<00:00, 149.67it/s]


Train MSE:  527330370.0   

  3%|▎         | 1/30 [00:29<14:01, 29.01s/it]

...

Val acc:  19159734.0
Epoch 29 of 30


100%|██████████| 3773/3773 [00:25<00:00, 149.11it/s]


Train MSE:  21621932.0   

100%|██████████| 30/30 [14:39<00:00, 29.33s/it]

Val acc:  14135380.0


In [103]:
ask_model.eval()
ask_model(torch.tensor(val_ask[:10]).float().unsqueeze(0).unsqueeze(2).cuda())

tensor(18955.6992, device='cuda:0', grad_fn=<SqueezeBackward0>)

In [104]:
val_ask[10]

22767.0

Моделька очень сильно ошибается, никуда не годится. Возможно стоило подумать в сторону RL методов.